# Summary of output from model of COVID-19 <a class="anchor" id="home"></a>

### Overview

This Jupyter notebook summarises output from the [OpenABM-Covid19](https://github.com/BDI-pathogens/OpenABM-Covid19) model.  This notebook provides basic plotting functions for a single simulation of the model using a single set of parameters.  The notebook is intended as a guide for how model output may be explored.  The figures in this notebook demonstrate some of the more detailed outputs from the model including 1) the time series output, 2) summary of all simulated SARS-CoV2 transmission events in a simulated epidemic, and 3) and all interaction events between individuals on the last day of the simulated outbreak.  Model output can be generated using this notebook by evaluating the cells below.  Four larger files generated by the model will be written to file in the location specified in the variable `output_dir` below.  These files are: 

1. `individual_file_Run1.csv`: a file of all individuals in the simulated epidemic.  
2. `transmission_Run1.csv`: a file of all transmission events in the simulated epidemic.  
3. `interactions_Run1.csv`: a file of all interaction events on the final day of the simulation.  
4. `trace_tokens_Run1.csv`: a file of all contact tracing events (up to the last 14 days) that have not been released.


The figures in this notebook are not intended to be used as projections of the current COVID19 outbreak.


### Usage

1. Download or clone the [OpenABM-Covid19](https://github.com/BDI-pathogens/OpenABM-Covid19) model.  
2. Install the Python interface for OpenABM-Covid19 (see the README.md for instructions).  
3. Evaluate each of the cells below.  


### Table of contents

*Output from a single simulation*
* [Parameter values and assumptions](#parameter-values-assumptions)
* [Time series](#time-series)
* [Interactions between individuals](#interactions)
* [Transmission network](#transmission-network)
* [Age-stratified outputs](#age-stratified-outputs)


*Notes*

This notebook was generated using the [OpenABM-Covid19](https://github.com/BDI-pathogens/OpenABM-Covid19) codebase 25th May 2020.  

In [ ]:
%matplotlib inline
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from os.path import join
from IPython.display import display
import plotting

### Model parameters

In [ ]:
from COVID19.model import Model, Parameters, ModelParameterException
import COVID19.simulation as simulation

input_parameter_file = "../tests/data/baseline_parameters.csv"
parameter_line_number = 1
output_dir = "results"
household_demographics_file = "../tests/data/baseline_household_demographics.csv"

### Run model

Simulate an epidemic with no interventions for 150 days using a population of 100,000.  

In [ ]:
params = Parameters(
    input_parameter_file, 
    parameter_line_number, 
    output_dir, 
    household_demographics_file)

T = 150
N = 100000
params.set_param("end_time", T)
params.set_param("n_total", N)

model = simulation.COVID19IBM(model = Model(params))
sim = simulation.Simulation(env = model, end_time = T)
sim.steps(T)

### Read input parameter file

In [ ]:
df_parameters = pd.read_csv(input_parameter_file)

# In case any parameter values have been redefined in this Jupyter notebook,
# read parameter values from Parameters object
df_parameters_used = plotting.get_df_from_params(params, df_parameters.columns)

### Write/import output data

In [ ]:
# Save timeseries data to pandas DataFrame
df_timeseries = pd.DataFrame(sim.results)

# Write extended output to file
sim.env.model.write_output_files()

# Import file with information on all individuals
df_indiv = pd.read_csv(join(output_dir, "individual_file_Run1.csv"))

# Import transmission file
df_trans = pd.read_csv(join(output_dir, "transmission_Run1.csv"))

# Import data on interactions in the last day of the simulation
df_interact = pd.read_csv(join(output_dir, "interactions_Run1.csv"))

### Define plotting constants

In [ ]:
from COVID19.model import AgeGroupEnum, EVENT_TYPES, TransmissionTypeEnum

interaction_types = [c.value for c in TransmissionTypeEnum ]
interaction_labels = [c.name[1:].title() for c in TransmissionTypeEnum]

infectious_compartments = ["PRESYMPTOMATIC", "PRESYMPTOMATIC_MILD", "ASYMPTOMATIC", "SYMPTOMATIC", "SYMPTOMATIC_MILD"]
infectious_types = [e.value for e in EVENT_TYPES if e.name in infectious_compartments]
infectious_names = [e.name for e in EVENT_TYPES if e.name in infectious_compartments]
infectious_labels = [plotting.EVENT_TYPE_STRING[e.value] for e in EVENT_TYPES if e.name in infectious_compartments]

# Define age groups and labels <- FIXME: change to Enum in constant.py
n_age = len(AgeGroupEnum) + 1
age_group_labels = [enum.name[1:].replace("_","-") for enum in AgeGroupEnum]
age_group_labels[-1] = "80+"

# Parameter values and assumptions <a class="anchor" id="parameter-values-assumptions"></a>
[[Home](#home)]

Key parameter values for the simulations below.  

In [ ]:
df_key_params = pd.DataFrame({
    "Parameter name": plotting.key_params,
    "Parameter value": [params.get_param(p) for p in plotting.key_params]
})
display(df_key_params.style.hide_index().set_precision(4))

## Waiting time distributions

In [ ]:
plt.rcParams['figure.figsize'] = [14, 12]
fig, ax = plotting.plot_parameter_assumptions(df_parameters_used)

## Age-specific parameters

In [ ]:
age_cols = [
    plotting.susceptibility_cols, plotting.asymptomatic_cols,
    plotting.hospitalised_cols, plotting.critical_cols,
    plotting.fatality_cols, plotting.mild_cols,
    plotting.app_users_cols
]

age_params = [[params.get_param(p) for p in cols] for cols in age_cols]
df = pd.DataFrame(age_params); df.columns = age_group_labels
df.rename(index = {0:'Relative susceptibility', 
                   1:'Pr( asymptomatic | infected )',
                   2:'Pr( hospitalisation | symptomatic )',
                   3:'Pr( ICU | hospitalised )',
                   4:'Pr( death | ICU )',
                   5:'Pr( mild | infected )',
                   6:'Pr( app users )'
                  }, inplace = True)
display(df.style.set_precision(2))

# Time series <a class="anchor" id="time-series"></a>
[[Home](#home)]

Plot the epidemic trajectory and other outputs from the time series file for a single parameter set.  

In [ ]:
plt.rcParams['figure.figsize'] = [14, 12]
fig, ax = plotting.plot_timeseries_curves(df_timeseries, xlimits = [0, 200])

# Interactions <a class="anchor" id="interactions"></a>
[[Home](#home)]

Summaries of interactions in the final day of the simulation.  


In [ ]:
plt.rcParams['figure.figsize'] = [14, 10]
# Count interactions for each individual by "type" of interaction
df_agg = df_interact.groupby(["ID_1", "type"])["ID_1"].count().reset_index(name = "count")

# Plot histogram of interaction by network type
fig, ax = plotting.plot_hist_by_group(df_agg, groupvar = "type", binvar = "count", 
    groups = interaction_types, bins = 25, group_colours = plotting.network_colours, 
    group_labels = interaction_labels, 
    xlabel = "Number of daily connections", 
    title = "Number of daily interactions by network type", 
    legend_title = "Network type")

In [ ]:
plt.rcParams['figure.figsize'] = [14, 10]
fig, ax = plotting.plot_interactions_by_age(
    df_interact, groupvar = "age_group_1", 
    group_labels = age_group_labels, legend_title = "Age group", 
    title = "Number of daily interactions by age group", 
    xlabel = "Number of daily connections", ylabel = "Count")


# Transmission network <a class="anchor" id="transmission-network"></a>
[[Home](#home)]


### Proportion of all transmission from infectious status of source

In [ ]:
# Remove seed cases
df_trans = df_trans[df_trans["status_source"] != EVENT_TYPES.SUSCEPTIBLE.value]

# Calculate percent transmissions by disease status of the source
percent_trans_status_source = 100*df_trans["status_source"].value_counts()/df_trans.shape[0]
percent_trans_status_source = percent_trans_status_source.reset_index()

# Summarise DataFrame
percent_trans_status_source.columns = ["status_source", "Percent transmissions"]
percent_trans_status_source["name"] = percent_trans_status_source.status_source.map(lambda x: plotting.EVENT_TYPE_STRING[EVENT_TYPES(x).value])

display(percent_trans_status_source[percent_trans_status_source["Percent transmissions"] > 0][["Percent transmissions", "name"]].style.hide_index().set_precision(2))

### Generation time by infection status of source

In [ ]:
plt.rcParams['figure.figsize'] = [14, 8]
fig, ax = plotting.plot_stacked_hist_by_group(df_trans, 
    groupvar = "status_source", binvar = "generation_time", 
    groups = infectious_types,
    NBINS = 18, group_labels = infectious_labels,
    xlabel = "Generation time", ylabel = "Count")

### Transmission events by age of infected individual and location of infection event

In [ ]:
plt.rcParams['figure.figsize'] = [16, 10]
fig, ax = plotting.plot_hist_by_group(df_trans[df_trans.infector_network >= 0], 
    groupvar = "infector_network", binvar = "age_group_recipient", 
    groups = [0, 1, 2], bins = n_age, group_colours = plotting.network_colours,
    group_labels = interaction_labels, 
    xlabel = "Age group of infected individual",ylabel = "Count",
    title = "Transmission events by age and network type",
    legend_title = "Network type", xticklabels = age_group_labels)

### Transmission events stratified by age

Of all transmission events, the following figure shows the distribution of transmission events (2D histogram) stratified by the age of the source and recipient.  

In [ ]:
fig, ax = plotting.plot_transmission_heatmap_by_age(df_trans, "age_group_recipient", "age_group_source", 
    bins = n_age, xlabel = "Age of recipient", ylabel = "Age of source", legend_title = "Number of transmission events",
    xticklabels = age_group_labels, yticklabels = age_group_labels)

In [ ]:
plt.rcParams['figure.figsize'] = [16, 8]
fig, ax = plotting.transmission_heatmap_by_age_by_panels(
    df_trans, "age_group_recipient", "age_group_source", bins = n_age, 
    panelvar = "infector_network", panel_labels = interaction_labels,
    xlabel = "Age of recipient", ylabel = "Age of source", 
    legend_title = "Number of\ntransmission events",
    xticklabels = age_group_labels, yticklabels = age_group_labels)

# Age-stratified outputs <a class="anchor" id="age-stratified-outputs"></a>

States that an individual may be in, stratified by age (of all individuals ever in such a state in the simulation).  

[[Home](#home)]

In [ ]:
plt.rcParams['figure.figsize'] = [16, 16]
groupvars = ["time_infected", "time_recovered", "time_death"]
labels = ["Infected", "Recovered", "Deaths"]

fig, ax = plotting.PlotHistByAge(df_trans, groupvars = groupvars, group_labels = labels,
    NBINS = n_age -1 , density = True, xticklabels = age_group_labels, xlabel = "Age group",
    ylim = 0.5, age_group_var = "age_group_recipient")

In [ ]:
plt.rcParams['figure.figsize'] = [16, 16]
groupvars = ["time_asymptomatic", "time_presymptomatic_mild", "time_presymptomatic_severe", "time_symptomatic_mild", "time_symptomatic_severe"]
labels = ["Asymptomatic", "Pre-symptomatic mild", "Pre-symptomatic severe", "Symptomatic mild", "Symptomatic severe"]
fig, ax = plotting.PlotHistByAge(df_trans, groupvars = groupvars, group_labels = labels,
    NBINS = n_age -1 , density = True, xticklabels = age_group_labels, xlabel = "Age group",
    ylim = 0.25, age_group_var = "age_group_recipient")

In [ ]:
plt.rcParams['figure.figsize'] = [16, 16]
groupvars = ["time_hospitalised", "time_critical"]
labels = ["Hospitalisations", "ICU"]

fig, ax = plotting.PlotHistByAge(df_trans, groupvars = groupvars, group_labels = labels,
    NBINS = n_age -1 , density = True, xticklabels = age_group_labels, xlabel = "Age group",
    ylim = 0.5, age_group_var = "age_group_recipient")

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
fig, ax = plotting.PlotHistIFRByAge(df_trans, "time_death", "time_infected", NBINS = n_age -1, 
    xticklabels = age_group_labels, xlabel = "Age group", age_group_var = "age_group_recipient")